In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
import openai 
openai.api_key = os.getenv("OPENAI_API_KEY")
from openai import OpenAI
client = OpenAI()
from fol_parser import parse_text_FOL_to_tree
import prompts

def gpt_call(prompt):
    """Calls on GPT-4 with the provided prompt.

    Args:
        prompt (list): The prompt to be used

    Returns:
        str: The response content from GPT-4
    """
    response =  openai.chat.completions.create(
        model = "gpt-4",
        temperature = 1.0,
        messages = prompt)
    return response

def gpt_prompt_fine_tune(prompt):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0613:personal::8hxFsIZp", 
        messages=prompt
    )
    return(response.choices[0].message.content)


def gpt_adjustment_fine_tune(prompt):
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0613:personal::8fluRAzK", 
        messages=prompt
    )
    return(response.choices[0].message.content)

def check_fol_val(f):
    """Uses the fol_parser to check if a FOL formula is valid.

    Args:
        f (str): The formula to be checked

    Returns:
        int: 0 if invalid, 1 if valid
    """
    tree = parse_text_FOL_to_tree(f)
    if tree == None:
        print(f'Formula: {f}\nINVALID')
        return 0
    else:
        print(f'Formula: {f}\nVALID')
        return 1

def nl_to_fol(i, prompt_function):
    """Iterates through a dataframe and calls for gpt to translate each sentence.
    Evaluates each translation as valid (1) or invalid (0), in addition to errors (-1).
    Saves the translations and evaluation to the file containing all translations.

    Args:
        df (DataFrame): The dataframe to iterate through
        prompt_iteration (str): Name of the prompt iteration
        save_file (str, optional): Filepath to save valid translations. Defaults to "".
        discard_file (str, optional): Filepath to save invalid translations. Defaults to "".
    """
    formulas = []
    evals = []
    prompt = prompt_function(i)
    try:
        formula = gpt_call(prompt)
    except:
        formulas.append('ERROR')
        evals.append(-1)
    if check_fol_val(formula) == 1:
        formulas.append(formula)
        evals.append(1)
    else:
        formulas.append(formula)
        evals.append(0)
    return formulas, evals
    # save_metadata(df, prompt_iteration)


def nl_to_fol_adjustment(s, f, prompt_iteration, adjustment):
    """Iterates through a dataframe and calls for gpt to adjust each sentence.
    Evaluates each translation as valid (1) or invalid (0), in addition to errors (-1).
    Saves the translations and evaluation to the file containing all translations.

    Args:
        df (DataFrame): The dataframe to iterate through
        prompt_iteration (str): Name of the prompt iteration
    """
    formulas = []
    evals = []
    prompt = adjustment(sentence=s, formula=f) 
    try:
        formula = gpt_call(prompt) 
    except:
        formulas.append('ERROR')
        evals.append(-1)
    if check_fol_val(formula) == 1:
        formulas.append(formula)
        evals.append(1)
    else:
        formulas.append(formula)
        evals.append(0)
    return formulas, evals

def cot(s, prompt_function, p=""):
    prompt = prompt_function(sentence=s, prev=p) 
    try:
        formula = gpt_call(prompt) 
        return formula
    except:
        print('ERROR')


prompt_function = prompts.prompt_10_BEST
prompt_iteration = "prompt_10_BEST"
adjustment_function = prompts.adjustment_prompt_10_BEST 
adjustment_iteration = "adjustment_prompt_10_BEST"


sentence = "it is polite to criticize someone else's parenting."

fol = nl_to_fol(sentence, prompt_function)
print(nl_to_fol_adjustment(sentence, fol, prompt_iteration, adjustment_function))

Formula: ∀x∀y∃z(Parenting(z,y) ∧ Criticize(x,y) → evaluation(POLITE))
VALID
Formula: ∀x∀y∃z(Parenting(z,y) ∧ Criticize(x,z) → evaluation(POLITE))
VALID
(['∀x∀y∃z(Parenting(z,y) ∧ Criticize(x,z) → evaluation(POLITE))'], [1])


## FOL TO CNF

In [1]:
from juliacall import Main as jl
jl.include("/Users/emmabjorkas/Documents/Informasjonsvitenskap/Master/gpt_pipeline/First-order-Logic-resolution-master/hw2.jl")

def replace_op(formula):
    """
    Replaces the operators in a formula to woek with Julia code.

    Args:
        formula (str): The formula to be edited

    Returns:
        str: The edited formula
    """
    formula = formula.replace('∨', '|')
    formula = formula.replace('∧', '&')
    formula = formula.replace('→', '==>')
    formula = formula.replace('↔', '<=>')
    formula = formula.replace('¬', '~')
    return formula


def fol_to_cnf_converter(lst):
    """Converts a list of FOL formulas to CNF using Julia code.

    Args:
        lst (list): The list of FOL formulas to be converted

    Returns:
        lst: The list of converted formulas.
    """
    clauses = jl.map(jl.skolemize, jl.map(jl.toCNF, jl.map(jl.toClause, jl.map(jl.lexer, lst))))
    #clauses = jl.map(jl.toCNF,(jl.map(jl.toClause, jl.map(jl.lexer, lst))))
    translated_formulas = []
    for i in clauses:
        #formatted = (i)
        formatted = jl.printClause(i)
        translated_formulas.append(formatted)
    return translated_formulas


def fol_to_cnf(lst, col_name="", prompt_iteration="", adjustment_iteration=""):
    """Creates a data frame of the CNF translations and evaluations.

    Args:
        df (DataFrame): The dataframe to iterate through
        col_name (str): Name of the column to save the translations and evaluations to
        prompt_iteration (str): Name of the prompt iteration
        adjustment_iteration (str): Name of the prompt iteration
    """

    cnf_formulas = []
    cnf_evals = []

    for index in range(len(lst)):

        converted = fol_to_cnf_converter([replace_op(lst[index])])
        print(converted)
        

lst = ["∀x∀y¬∃z∀q(Asked(z) ∧ Stanger(x) ∧ Advice(q) ∧ GiveTo(y,q,x) → evaluation(RUDE))"]
fol_to_cnf(lst)

['evaluation(RUDE) | ~Asked(v3) | ~Stanger(v0) | ~Advice(v2) | ~GiveTo(v1, v2, v0)']
